In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Function to load and display basic dataset information
def load_and_explore_data(file_path):
    """
    Load the dataset and perform initial exploration
    """
    print(f"Loading data from {file_path}...")
    df = pd.read_csv(file_path)
    
    print(f"\nDataset shape: {df.shape}")
    print(f"Number of columns: {len(df.columns)}")
    
    # Sample data
    print("\nSample data (first 5 rows):")
    print(df.head())
    
    # Data types
    print("\nData types:")
    print(df.dtypes.value_counts())
    
    # Check for missing values
    missing_values = df.isnull().sum()
    missing_percent = (missing_values / len(df)) * 100
    missing_data = pd.DataFrame({
        'Missing Values': missing_values,
        'Percentage': missing_percent
    })
    print("\nTop 20 columns with missing values:")
    print(missing_data[missing_data['Missing Values'] > 0].sort_values('Missing Values', ascending=False).head(20))
    
    return df

# Function to convert date/timestamp columns
def convert_date_columns(df):
    """
    Convert date and timestamp columns to proper datetime format
    """
    print("\nConverting date columns...")
    # Identify potential date columns (by name or data type)
    date_columns = [col for col in df.columns if 'Date' in col or 'Time' in col or 'Timestamp' in col]
    
    for col in date_columns:
        if col in df.columns:
            try:
                df[col] = pd.to_datetime(df[col])
                print(f"Converted {col} to datetime")
            except Exception as e:
                print(f"Could not convert {col} to datetime: {e}")
    
    return df

# Function to analyze numerical distributions
def analyze_numerical_columns(df):
    """
    Analyze the distribution of numerical columns
    """
    print("\nAnalyzing numerical columns...")
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    # Basic statistics
    print("\nBasic statistics for numerical columns:")
    print(df[numerical_cols].describe().T)
    
    # Create distribution plots for key financial columns
    financial_cols = ['Price', 'Qty_', 'Sub_Total', 'Discount', 'Tax', 'Final_Total']
    financial_cols = [col for col in financial_cols if col in numerical_cols]
    
    if financial_cols:
        fig, axes = plt.subplots(len(financial_cols), 2, figsize=(15, 4*len(financial_cols)))
        
        for i, col in enumerate(financial_cols):
            # Histogram
            sns.histplot(df[col].dropna(), ax=axes[i, 0])
            axes[i, 0].set_title(f'Distribution of {col}')
            axes[i, 0].set_xlabel(col)
            
            # Box plot
            sns.boxplot(x=df[col].dropna(), ax=axes[i, 1])
            axes[i, 1].set_title(f'Boxplot of {col}')
            axes[i, 1].set_xlabel(col)
        
        plt.tight_layout()
        plt.savefig('numerical_distributions.png')
        plt.close()
        print("Saved distribution plots to 'numerical_distributions.png'")
    
    return numerical_cols

# Function to analyze categorical columns
def analyze_categorical_columns(df):
    """
    Analyze categorical columns distributions
    """
    print("\nAnalyzing categorical columns...")
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    # Value counts for selected categorical columns
    important_cat_cols = ['Status', 'Payment_Type', 'Order_Type', 'Area']
    important_cat_cols = [col for col in important_cat_cols if col in categorical_cols]
    
    for col in important_cat_cols:
        print(f"\nValue counts for {col}:")
        counts = df[col].value_counts()
        print(counts.head(10))  # Show top 10 values
        
        # Create bar plots
        plt.figure(figsize=(10, 6))
        counts.head(10).plot(kind='bar')
        plt.title(f'Top 10 values for {col}')
        plt.xlabel(col)
        plt.ylabel('Count')
        plt.tight_layout()
        plt.savefig(f'{col}_distribution.png')
        plt.close()
        print(f"Saved {col} distribution to '{col}_distribution.png'")
    
    return categorical_cols

# Function to check for data consistency
def check_data_consistency(df):
    """
    Check for data consistency across related fields
    """
    print("\nChecking data consistency...")
    
    # Check if subtotal matches price * quantity
    if all(col in df.columns for col in ['Price', 'Qty_', 'Sub_Total']):
        df['calculated_subtotal'] = df['Price'] * df['Qty_']
        df['subtotal_diff'] = np.abs(df['calculated_subtotal'] - df['Sub_Total'])
        subtotal_mismatch = df[df['subtotal_diff'] > 0.01]
        
        print(f"Records with Price * Qty ≠ Sub_Total: {len(subtotal_mismatch)} ({len(subtotal_mismatch)/len(df)*100:.2f}%)")
        if len(subtotal_mismatch) > 0:
            print("Sample mismatches:")
            print(subtotal_mismatch[['Price', 'Qty_', 'calculated_subtotal', 'Sub_Total', 'subtotal_diff']].head())
    
    # Check if final total matches subtotal - discount + tax
    if all(col in df.columns for col in ['Sub_Total', 'Discount', 'Tax', 'Final_Total']):
        df['calculated_final'] = df['Sub_Total'] - df['Discount'] + df['Tax']
        df['final_diff'] = np.abs(df['calculated_final'] - df['Final_Total'])
        final_mismatch = df[df['final_diff'] > 0.01]
        
        print(f"Records with Sub_Total - Discount + Tax ≠ Final_Total: {len(final_mismatch)} ({len(final_mismatch)/len(df)*100:.2f}%)")
        if len(final_mismatch) > 0:
            print("Sample mismatches:")
            print(final_mismatch[['Sub_Total', 'Discount', 'Tax', 'calculated_final', 'Final_Total', 'final_diff']].head())
    
    # Check tax calculation consistency
    if all(col in df.columns for col in ['CGST_Amount', 'SGST_Amount', 'Tax']):
        df['calculated_tax'] = df['CGST_Amount'] + df['SGST_Amount']
        if 'VAT_Amount' in df.columns:
            df['calculated_tax'] += df['VAT_Amount']
        if 'Service_Charge_Amount' in df.columns:
            df['calculated_tax'] += df['Service_Charge_Amount']
        
        df['tax_diff'] = np.abs(df['calculated_tax'] - df['Tax'])
        tax_mismatch = df[df['tax_diff'] > 0.01]
        
        print(f"Records with tax component sum ≠ Tax: {len(tax_mismatch)} ({len(tax_mismatch)/len(df)*100:.2f}%)")
        if len(tax_mismatch) > 0:
            print("Sample mismatches:")
            print(tax_mismatch[['CGST_Amount', 'SGST_Amount', 'calculated_tax', 'Tax', 'tax_diff']].head())
    
    # Compare status fields across systems
    if all(col in df.columns for col in ['Status', 'Status_z']):
        status_z_mismatch = df[(~df['Status_z'].isna()) & (df['Status'] != df['Status_z'])]
        print(f"Records with Status ≠ Status_z: {len(status_z_mismatch)} ({len(status_z_mismatch)/len(df)*100:.2f}%)")
    
    if all(col in df.columns for col in ['Status', 'Status_s']):
        status_s_mismatch = df[(~df['Status_s'].isna()) & (df['Status'] != df['Status_s'])]
        print(f"Records with Status ≠ Status_s: {len(status_s_mismatch)} ({len(status_s_mismatch)/len(df)*100:.2f}%)")
    
    # Check for invoice modifications
    if all(col in df.columns for col in ['amount_from', 'amount_to']):
        modified_invoices = df[(~df['amount_from'].isna()) & (~df['amount_to'].isna())]
        print(f"Modified invoices: {len(modified_invoices)} ({len(modified_invoices)/len(df)*100:.2f}%)")
        
        if len(modified_invoices) > 0:
            df.loc[modified_invoices.index, 'modification_amount'] = df.loc[modified_invoices.index, 'amount_to'] - df.loc[modified_invoices.index, 'amount_from']
            print("Modification amount statistics:")
            print(df.loc[modified_invoices.index, 'modification_amount'].describe())
    
    return df

# Function for feature engineering
def engineer_features(df):
    """
    Create derived features that might help detect anomalies
    """
    print("\nEngineering features...")
    
    # Financial calculation features
    if all(col in df.columns for col in ['Price', 'Qty_', 'Sub_Total']):
        df['price_qty_match'] = np.isclose(df['Price'] * df['Qty_'], df['Sub_Total'], rtol=0.01)
        print(f"Records with price * quantity matching subtotal: {df['price_qty_match'].sum()} ({df['price_qty_match'].mean()*100:.2f}%)")
    
    if all(col in df.columns for col in ['Sub_Total', 'Discount', 'Tax', 'Final_Total']):
        df['formula_match'] = np.isclose(df['Sub_Total'] - df['Discount'] + df['Tax'], df['Final_Total'], rtol=0.01)
        print(f"Records with subtotal - discount + tax matching final total: {df['formula_match'].sum()} ({df['formula_match'].mean()*100:.2f}%)")
    
    # Tax calculation features
    tax_components = ['CGST_Amount', 'SGST_Amount', 'VAT_Amount', 'Service_Charge_Amount']
    available_components = [col for col in tax_components if col in df.columns]
    
    if available_components and 'Tax' in df.columns:
        df['calculated_tax'] = df[available_components].sum(axis=1)
        df['tax_matches'] = np.isclose(df['calculated_tax'], df['Tax'], rtol=0.01)
        print(f"Records with tax components matching total tax: {df['tax_matches'].sum()} ({df['tax_matches'].mean()*100:.2f}%)")
    
    # Discount percentage feature
    if all(col in df.columns for col in ['Discount', 'Sub_Total']):
        df['discount_percentage'] = (df['Discount'] / df['Sub_Total']) * 100
        df.loc[df['Sub_Total'] == 0, 'discount_percentage'] = 0
        print("Discount percentage statistics:")
        print(df['discount_percentage'].describe())
        
        # Flag high discounts
        df['high_discount'] = df['discount_percentage'] > 50
        print(f"Records with discount > 50%: {df['high_discount'].sum()} ({df['high_discount'].mean()*100:.2f}%)")
    
    # Invoice modification features
    if all(col in df.columns for col in ['amount_from', 'amount_to']):
        df['invoice_modified'] = (~df['amount_from'].isna()) & (~df['amount_to'].isna())
        df.loc[df['invoice_modified'], 'modification_amount'] = df.loc[df['invoice_modified'], 'amount_to'] - df.loc[df['invoice_modified'], 'amount_from']
        df.loc[df['invoice_modified'], 'modification_percentage'] = (df.loc[df['invoice_modified'], 'modification_amount'] / df.loc[df['invoice_modified'], 'amount_from']) * 100
        
        # Flag significant reductions
        df['significant_reduction'] = (df['invoice_modified']) & (df['modification_amount'] < -10)
        print(f"Records with significant price reduction (>10): {df['significant_reduction'].sum()}")
    
    # Timing-related features
    time_pairs = []
    if all(col in df.columns for col in ['Received_Time_z', 'Delivered_Time_z']):
        time_pairs.append(('Received_Time_z', 'Delivered_Time_z', 'zomato_delivery_minutes'))
    
    if all(col in df.columns for col in ['Received_Time_s', 'Delivered_Time_s']):
        time_pairs.append(('Received_Time_s', 'Delivered_Time_s', 'swiggy_delivery_minutes'))
    
    for start_col, end_col, result_col in time_pairs:
        # Calculate time difference in minutes
        valid_times = (~df[start_col].isna()) & (~df[end_col].isna())
        if valid_times.sum() > 0:
            df.loc[valid_times, result_col] = (df.loc[valid_times, end_col] - df.loc[valid_times, start_col]).dt.total_seconds() / 60
            print(f"{result_col} statistics:")
            print(df.loc[valid_times, result_col].describe())
            
            # Flag unrealistic timing
            df[f'{result_col}_suspicious'] = (df[result_col] < 10) | (df[result_col] > 120)
            suspicious_count = df[f'{result_col}_suspicious'].sum()
            print(f"Records with suspicious {result_col} (< 10 min or > 120 min): {suspicious_count}")
    
    # Status inconsistency features
    if all(col in df.columns for col in ['Status', 'Status_z']):
        df['status_z_mismatch'] = (~df['Status_z'].isna()) & (df['Status'] != df['Status_z'])
        mismatch_count = df['status_z_mismatch'].sum()
        print(f"Status vs Status_z mismatches: {mismatch_count}")
    
    if all(col in df.columns for col in ['Status', 'Status_s']):
        df['status_s_mismatch'] = (~df['Status_s'].isna()) & (df['Status'] != df['Status_s'])
        mismatch_count = df['status_s_mismatch'].sum()
        print(f"Status vs Status_s mismatches: {mismatch_count}")
    
    # Cancelled with charges feature
    if 'Status' in df.columns and 'Final_Total' in df.columns:
        df['cancelled_with_charges'] = (df['Status'] == 'Cancelled') & (df['Final_Total'] > 0)
        cancelled_count = df['cancelled_with_charges'].sum()
        print(f"Cancelled orders with charges: {cancelled_count}")
    
    return df

# Function to identify potential anomalies using basic rules
def identify_basic_anomalies(df):
    """
    Identify potential anomalies using basic rule-based detection
    """
    print("\nIdentifying potential anomalies using basic rules...")
    
    anomaly_flags = []
    
    # Define anomaly conditions
    if 'price_qty_match' in df.columns:
        anomaly_flags.append(('price_qty_mismatch', ~df['price_qty_match']))
    
    if 'formula_match' in df.columns:
        anomaly_flags.append(('total_calculation_error', ~df['formula_match']))
    
    if 'tax_matches' in df.columns:
        anomaly_flags.append(('tax_calculation_error', ~df['tax_matches']))
    
    if 'high_discount' in df.columns:
        anomaly_flags.append(('high_discount', df['high_discount']))
    
    if 'significant_reduction' in df.columns:
        anomaly_flags.append(('significant_price_reduction', df['significant_reduction']))
    
    time_columns = [col for col in df.columns if col.endswith('_suspicious')]
    for col in time_columns:
        anomaly_flags.append((col.replace('_suspicious', '_anomaly'), df[col]))
    
    if 'status_z_mismatch' in df.columns:
        anomaly_flags.append(('zomato_status_mismatch', df['status_z_mismatch']))
    
    if 'status_s_mismatch' in df.columns:
        anomaly_flags.append(('swiggy_status_mismatch', df['status_s_mismatch']))
    
    if 'cancelled_with_charges' in df.columns:
        anomaly_flags.append(('cancelled_with_charges', df['cancelled_with_charges']))
    
    # Add flags to dataframe
    for flag_name, condition in anomaly_flags:
        df[flag_name] = condition
        anomaly_count = condition.sum()
        print(f"{flag_name}: {anomaly_count} records ({anomaly_count/len(df)*100:.2f}%)")
    
    # Create overall anomaly flag
    all_flag_columns = [flag for flag, _ in anomaly_flags]
    if all_flag_columns:
        df['any_basic_anomaly'] = df[all_flag_columns].any(axis=1)
        anomaly_count = df['any_basic_anomaly'].sum()
        print(f"\nTotal records with at least one anomaly: {anomaly_count} ({anomaly_count/len(df)*100:.2f}%)")
    
    return df

# Main execution function
def analyze_and_preprocess_data(file_path):
    """
    Main function to analyze and preprocess data
    """
    # Load data
    df = load_and_explore_data(file_path)
    
    # Convert date columns
    df = convert_date_columns(df)
    
    # Analyze numerical columns
    numerical_cols = analyze_numerical_columns(df)
    
    # Analyze categorical columns
    categorical_cols = analyze_categorical_columns(df)
    
    # Check data consistency
    df = check_data_consistency(df)
    
    # Engineer features
    df = engineer_features(df)
    
    # Identify basic anomalies
    df = identify_basic_anomalies(df)
    
    # Save processed data
    processed_file = 'processed_data.csv'
    df.to_csv(processed_file, index=False)
    print(f"\nSaved processed data to {processed_file}")
    
    # Return the processed dataframe and column info
    return df, numerical_cols, categorical_cols

# Run the analysis if executed directly
if __name__ == "__main__":
    # Replace 'hackathon_data.csv' with your actual file path
    df, numerical_cols, categorical_cols = analyze_and_preprocess_data('/kaggle/input/hakathon/Hackathon/Hackathon Dataset.csv')
    
    # Print summary
    print("\n--- Data Analysis Summary ---")
    print(f"Total records: {len(df)}")
    print(f"Numerical columns: {len(numerical_cols)}")
    print(f"Categorical columns: {len(categorical_cols)}")
    if 'any_basic_anomaly' in df.columns:
        anomaly_count = df['any_basic_anomaly'].sum()
        print(f"Potential anomalies detected: {anomaly_count} ({anomaly_count/len(df)*100:.2f}%)")


Loading data from /kaggle/input/hakathon/Hackathon/Hackathon Dataset.csv...

Dataset shape: (205888, 161)
Number of columns: 161

Sample data (first 5 rows):
         Date                Timestamp Invoice_No_   Payment_Type Order_Type  \
0  13-10-2024  2024-10-13 22:42:25 UTC       11507           Cash    Dine In   
1  04-10-2024  2024-10-04 18:53:25 UTC       C1984           Cash    Dine In   
2  10-03-2025  2025-03-11 00:24:33 UTC       21337  Other [Paytm]    Dine In   
3  24-08-2024  2024-08-24 15:23:25 UTC        8455           CARD    Dine In   
4  20-01-2025  2025-01-20 22:12:29 UTC       18290           CARD    Dine In   

           Area                                    Item_Name  Price  Qty_  \
0  Garden Table                                 Basmati Rice    375   1.0   
1  Garden Table                  Exotic Stir Fried Vegetable    795   1.0   
2       Dine in                                    THUMPS UP    185   1.0   
3  Garden Table                           Manchow Sou

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import joblib
import time
import warnings
warnings.filterwarnings('ignore')


# Function to load preprocessed data
def load_preprocessed_data(file_path='processed_data.csv'):
    """
    Load the preprocessed data saved from the analysis phase
    """
    print("Loading preprocessed data...")
    df = pd.read_csv(file_path)
    print(f"Loaded {len(df)} records with {len(df.columns)} features")
    
    # Verify that anomaly flags are present
    anomaly_flags = [col for col in df.columns if 'anomaly' in col or 'mismatch' in col or col == 'high_discount' or col == 'significant_price_reduction' or col == 'cancelled_with_charges']
    print(f"Found {len(anomaly_flags)} anomaly flag columns: {anomaly_flags}")
    
    # Check for the aggregate anomaly flag
    if 'any_basic_anomaly' in df.columns:
        anomaly_count = df['any_basic_anomaly'].sum()
        print(f"Records with at least one anomaly: {anomaly_count} ({anomaly_count/len(df)*100:.2f}%)")
    
    return df, anomaly_flags


# Function to prepare features for modeling
def prepare_features(df):
    """
    Prepare features for modeling by selecting relevant columns
    and handling missing values
    """
    print("\nPreparing features for modeling...")
    
    # Select numerical features relevant for anomaly detection
    numerical_features = [
        'Price', 'Qty_', 'Sub_Total', 'Discount', 'Tax', 'Final_Total',
        'discount_percentage'
    ]
    
    # Add calculated fields if they exist
    optional_numericals = [
        'modification_amount', 'zomato_delivery_minutes', 'swiggy_delivery_minutes',
        'CGST_Amount', 'SGST_Amount', 'VAT_Amount', 'Service_Charge_Amount'
    ]
    
    for col in optional_numericals:
        if col in df.columns:
            numerical_features.append(col)
    
    # Select categorical features
    categorical_features = [
        'Status', 'Payment_Type', 'Order_Type'
    ]
    
    # Optional categorical features
    optional_categoricals = [
        'Area', 'Category'
    ]
    
    for col in optional_categoricals:
        if col in df.columns:
            categorical_features.append(col)
    
    # Remove features with too many missing values
    selected_features = []
    for feature in numerical_features + categorical_features:
        if feature in df.columns:
            missing_pct = df[feature].isna().mean() * 100
            if missing_pct < 30:  # Keep if less than 30% missing
                selected_features.append(feature)
                print(f"Selected {feature} (missing: {missing_pct:.2f}%)")
            else:
                print(f"Dropped {feature} due to high missingness: {missing_pct:.2f}%")
    
    # Create feature dataframe with selected columns
    features_df = df[selected_features].copy()
    
    # Create target variable from the anomaly flags
    if 'any_basic_anomaly' in df.columns:
        y = df['any_basic_anomaly']
    else:
        anomaly_flags = [col for col in df.columns if 'anomaly' in col or 'mismatch' in col or col == 'high_discount' or col == 'significant_price_reduction' or col == 'cancelled_with_charges']
        if anomaly_flags:
            y = df[anomaly_flags].any(axis=1)
        else:
            y = None
            print("Warning: No anomaly flags found, cannot create target variable")
    
    # Basic preprocessing
    numeric_cols = features_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_cols = features_df.select_dtypes(include=['object']).columns.tolist()
    
    print(f"\nSelected {len(numeric_cols)} numeric features and {len(categorical_cols)} categorical features")
    print(f"Numeric features: {numeric_cols}")
    print(f"Categorical features: {categorical_cols}")
    
    # Handle missing values
    for col in numeric_cols:
        features_df[col] = features_df[col].fillna(features_df[col].median())
    
    for col in categorical_cols:
        features_df[col] = features_df[col].fillna(features_df[col].mode()[0])
    
    return features_df, numeric_cols, categorical_cols, y


# Function to create preprocessor pipeline
def create_preprocessor(numeric_cols, categorical_cols):
    """
    Create a preprocessing pipeline for numeric and categorical features
    """
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])
    
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_cols),
            ('cat', categorical_transformer, categorical_cols)
        ]
    )
    
    return preprocessor


# Function to train and evaluate unsupervised anomaly detection models
def train_unsupervised_models(X, y_true=None):
    """
    Train and evaluate multiple unsupervised anomaly detection models
    """
    print("\nTraining unsupervised anomaly detection models...")
    
    # Define models to train
    models = {
        'Isolation Forest': IsolationForest(contamination=0.1, random_state=42),
        'One-Class SVM': OneClassSVM(nu=0.1, gamma='scale'),
        'Local Outlier Factor': LocalOutlierFactor(n_neighbors=20, contamination=0.1),
        'DBSCAN': DBSCAN(eps=0.5, min_samples=5)
    }
    
    results = {}
    for name, model in models.items():
        print(f"\nTraining {name}...")
        start_time = time.time()
        
        if name == 'DBSCAN':
            y_pred = model.fit_predict(X)
            # Convert DBSCAN labels to binary anomaly scores (-1 for anomalies, 1 for inliers)
            y_pred = np.where(y_pred == -1, -1, 1)
        elif name == 'Local Outlier Factor':
            y_pred = model.fit_predict(X)
        else:
            y_pred = model.fit_predict(X)
        
        training_time = time.time() - start_time
        
        # For evaluation consistency, ensure -1 is anomaly and 1 is normal
        y_pred_binary = np.where(y_pred == -1, 1, 0)  # Convert to 1 for anomaly, 0 for normal
        
        results[name] = {
            'model': model,
            'predictions': y_pred_binary,
            'training_time': training_time
        }
        
        # If true labels are provided, calculate performance metrics
        if y_true is not None:
            accuracy = accuracy_score(y_true, y_pred_binary)
            precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred_binary, average='binary')
            
            results[name].update({
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1': f1
            })
            
            print(f"Performance of {name}:")
            print(f"Accuracy: {accuracy:.4f}")
            print(f"Precision: {precision:.4f}")
            print(f"Recall: {recall:.4f}")
            print(f"F1 Score: {f1:.4f}")
            print(f"Training time: {training_time:.2f} seconds")
            
            # Confusion matrix
            cm = confusion_matrix(y_true, y_pred_binary)
            print("Confusion Matrix:")
            print(cm)
    
    return results


# Function to train and evaluate supervised model for severity classification
def train_supervised_model(X, y, features_df=None, severity_levels=None):
    """
    Train and evaluate a supervised model for anomaly severity classification
    """
    print("\nTraining supervised model for anomaly severity classification...")
    
    # If severity levels are not provided, use boolean classification
    if severity_levels is None:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y
        )
        
        # Train RandomForest classifier
        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)
        
        # Evaluate
        y_pred = rf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
        
        print(f"RandomForest Classifier Performance:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        
        # Feature importance - FIX: Using importances without feature_names_in_
        importances = rf.feature_importances_
        
        # Create feature names (since we don't have feature_names_in_)
        if features_df is not None:
            # Get feature names from the original dataframe
            feature_names = features_df.columns.tolist()
            # Ensure we don't have more importances than feature names (could happen with one-hot encoding)
            if len(importances) > len(feature_names):
                feature_names = [f'feature_{i}' for i in range(len(importances))]
        else:
            # If no features_df is provided, create generic feature names
            feature_names = [f'feature_{i}' for i in range(len(importances))]
            
        # Create importance dataframe
        feature_importance = pd.DataFrame({
            'Feature': feature_names[:len(importances)],  # Ensure we don't exceed the length
            'Importance': importances
        }).sort_values('Importance', ascending=False)
        
        print("\nTop 10 important features (estimated):")
        print(feature_importance.head(10))
        
        # Plot feature importance
        plt.figure(figsize=(12, 8))
        # Use only top 15 features for readability
        top_15 = feature_importance.head(15)
        sns.barplot(x='Importance', y='Feature', data=top_15)
        plt.title('Feature Importance (RandomForest)')
        plt.tight_layout()
        plt.savefig('feature_importance.png')
        plt.close()
        
        return {
            'model': rf,
            'feature_importance': feature_importance,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'predictions': y_pred,
            'test_indices': range(len(y_test))  # Simplified indices
        }
    else:
        # Multi-class classification for severity levels
        # Implementation would be similar but handling multiple classes
        pass


# Function to compare models
def compare_models(unsupervised_results, supervised_result=None):
    """
    Compare model performance and generate visualizations
    """
    print("\nComparing model performance...")
    
    # Extract performance metrics
    model_names = []
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    training_times = []
    
    for name, result in unsupervised_results.items():
        if 'accuracy' in result:
            model_names.append(name)
            accuracies.append(result['accuracy'])
            precisions.append(result['precision'])
            recalls.append(result['recall'])
            f1_scores.append(result['f1'])
            training_times.append(result['training_time'])
    
    if supervised_result and 'accuracy' in supervised_result:
        model_names.append('RandomForest (Supervised)')
        accuracies.append(supervised_result['accuracy'])
        precisions.append(supervised_result['precision'])
        recalls.append(supervised_result['recall'])
        f1_scores.append(supervised_result['f1'])
        # If supervised_result doesn't have training_time, add a placeholder
        if supervised_result.get('training_time'):
            training_times.append(supervised_result['training_time'])
        else:
            training_times.append(0)  # Placeholder value
    
    # Create comparison dataframe
    comparison_df = pd.DataFrame({
        'Model': model_names,
        'Accuracy': accuracies,
        'Precision': precisions,
        'Recall': recalls,
        'F1 Score': f1_scores,
        'Training Time (s)': training_times
    })
    
    print("\nModel Performance Comparison:")
    print(comparison_df)
    
    # Plot comparison
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    plt.figure(figsize=(15, 10))
    
    # Create subplots
    for i, metric in enumerate(metrics):
        plt.subplot(2, 2, i+1)
        bars = sns.barplot(x='Model', y=metric, data=comparison_df)
        plt.title(f'Model Comparison - {metric}')
        plt.xticks(rotation=45)
        
        # Add value labels on top of bars
        for j, bar in enumerate(bars.patches):
            bars.annotate(f'{comparison_df[metric].iloc[j]:.3f}',
                         (bar.get_x() + bar.get_width() / 2, bar.get_height()),
                         ha='center', va='bottom',
                         fontsize=8)
    
    plt.tight_layout()
    plt.savefig('model_comparison.png')
    plt.close()
    
    return comparison_df


# Function to identify and analyze anomalies
def analyze_anomalies(df, best_model_results, features_df, numeric_cols, categorical_cols, threshold=0.9):
    """
    Analyze the anomalies detected by the best model
    """
    print("\nAnalyzing detected anomalies...")
    
    # Get predictions from the best model
    y_pred = best_model_results['predictions']
    
    # Create a DataFrame with anomalies
    if isinstance(y_pred, np.ndarray) and len(y_pred) == len(df):
        # If predictions match the original dataframe length
        anomaly_indices = np.where(y_pred == 1)[0]
        anomalies_df = df.iloc[anomaly_indices].copy()
    else:
        # If predictions are from a test set, we need to handle this differently
        print("Warning: Predictions do not match dataframe length. Using a sample of top anomalies.")
        # Use any existing anomaly flags
        anomaly_flags = [col for col in df.columns if 'anomaly' in col or 'mismatch' in col or col == 'high_discount' or col == 'significant_price_reduction' or col == 'cancelled_with_charges']
        if 'any_basic_anomaly' in df.columns:
            anomalies_df = df[df['any_basic_anomaly'] == 1].sample(min(1000, df['any_basic_anomaly'].sum())).copy()
        elif anomaly_flags:
            anomalies_df = df[df[anomaly_flags].any(axis=1)].sample(min(1000, df[anomaly_flags].any(axis=1).sum())).copy()
        else:
            # Fall back to a random sample
            anomalies_df = df.sample(min(1000, len(df) // 100)).copy()
    
    print(f"Total anomalies detected: {len(anomalies_df)}")
    
    # Analyze anomaly types
    anomaly_flags = [col for col in df.columns if 'anomaly' in col or 'mismatch' in col or col == 'high_discount' or col == 'significant_price_reduction' or col == 'cancelled_with_charges']
    
    print("\nBreakdown of anomaly types:")
    for flag in anomaly_flags:
        if flag in anomalies_df.columns:
            count = anomalies_df[flag].sum()
            pct = count / len(anomalies_df) * 100
            print(f"{flag}: {count} ({pct:.2f}%)")
    
    # Analyze key characteristics of anomalies
    print("\nCharacteristics of anomalies:")
    
    for col in numeric_cols:
        if col in anomalies_df.columns and col in df.columns:
            anomaly_mean = anomalies_df[col].mean()
            normal_mean = df[~df.index.isin(anomalies_df.index)][col].mean()
            print(f"{col}: Anomaly mean = {anomaly_mean:.2f}, Normal mean = {normal_mean:.2f}")
    
    for col in categorical_cols:
        if col in anomalies_df.columns:
            print(f"\nDistribution of {col} in anomalies:")
            value_counts = anomalies_df[col].value_counts().head(5)
            print(value_counts)
    
    # Assign severity scores using a more meaningful approach
    # We'll use a combination of factors to determine severity
    
    # 1. Check for invoice modifications (if available)
    if 'modification_amount' in anomalies_df.columns:
        anomalies_df['mod_severity'] = 0
        # Higher severity for larger modifications
        cond1 = (anomalies_df['modification_amount'] < -1000) 
        cond2 = (anomalies_df['modification_amount'] < -500) & (anomalies_df['modification_amount'] >= -1000)
        cond3 = (anomalies_df['modification_amount'] < -100) & (anomalies_df['modification_amount'] >= -500)
        
        anomalies_df.loc[cond1, 'mod_severity'] = 3  # High
        anomalies_df.loc[cond2, 'mod_severity'] = 2  # Medium
        anomalies_df.loc[cond3, 'mod_severity'] = 1  # Low
    else:
        anomalies_df['mod_severity'] = 0
    
    # 2. Check for high discounts
    if 'discount_percentage' in anomalies_df.columns:
        anomalies_df['discount_severity'] = 0
        # Higher severity for larger discounts
        cond1 = (anomalies_df['discount_percentage'] > 50)
        cond2 = (anomalies_df['discount_percentage'] > 30) & (anomalies_df['discount_percentage'] <= 50)
        cond3 = (anomalies_df['discount_percentage'] > 20) & (anomalies_df['discount_percentage'] <= 30)
        
        anomalies_df.loc[cond1, 'discount_severity'] = 3  # High
        anomalies_df.loc[cond2, 'discount_severity'] = 2  # Medium
        anomalies_df.loc[cond3, 'discount_severity'] = 1  # Low
    else:
        anomalies_df['discount_severity'] = 0
    
    # 3. Check for cancelled orders with charges
    anomalies_df['cancel_severity'] = 0
    if 'cancelled_with_charges' in anomalies_df.columns and 'Final_Total' in anomalies_df.columns:
        cond1 = (anomalies_df['cancelled_with_charges'] == True) & (anomalies_df['Final_Total'] > 1000)
        cond2 = (anomalies_df['cancelled_with_charges'] == True) & (anomalies_df['Final_Total'] > 500) & (anomalies_df['Final_Total'] <= 1000)
        cond3 = (anomalies_df['cancelled_with_charges'] == True) & (anomalies_df['Final_Total'] <= 500)
        
        anomalies_df.loc[cond1, 'cancel_severity'] = 3  # High
        anomalies_df.loc[cond2, 'cancel_severity'] = 2  # Medium
        anomalies_df.loc[cond3, 'cancel_severity'] = 1  # Low
    
    # 4. Check for status mismatches
    anomalies_df['status_severity'] = 0
    if 'status_z_mismatch' in anomalies_df.columns or 'status_s_mismatch' in anomalies_df.columns:
        if 'status_z_mismatch' in anomalies_df.columns:
            anomalies_df.loc[anomalies_df['status_z_mismatch'] == True, 'status_severity'] = 2
        if 'status_s_mismatch' in anomalies_df.columns:
            anomalies_df.loc[anomalies_df['status_s_mismatch'] == True, 'status_severity'] = 2
    
    # Combine all severity factors to get overall severity score
    anomalies_df['severity_score'] = anomalies_df[['mod_severity', 'discount_severity', 'cancel_severity', 'status_severity']].max(axis=1)
    
    # If all are zero, assign a low severity
    anomalies_df.loc[anomalies_df['severity_score'] == 0, 'severity_score'] = 1
    
    # Map to severity categories
    severity_map = {1: 'Low', 2: 'Medium', 3: 'High'}
    anomalies_df['severity'] = anomalies_df['severity_score'].map(severity_map)
    
    print("\nSeverity distribution:")
    print(anomalies_df['severity'].value_counts())
    
    # Sample anomalies by severity
    if 'High' in anomalies_df['severity'].values:
        print("\nSample high severity anomalies:")
        high_severity = anomalies_df[anomalies_df['severity'] == 'High'].head(5)
        print(high_severity[['Invoice_No_', 'Status', 'Payment_Type', 'Sub_Total', 'Final_Total', 'discount_percentage']])
    
    # Return the anomalies with severity
    return anomalies_df


# Function to save trained models
def save_models(models_dict, filename='anomaly_detection_models.pkl'):
    """
    Save trained models to a file
    """
    joblib.dump(models_dict, filename)
    print(f"\nSaved models to {filename}")


# Function to create serializable anomaly data for API
def create_anomaly_data_for_api(anomalies_df, schema_info=None):
    """
    Create serializable data for the API
    """
    print("\nPreparing anomaly data for API...")
    
    # If schema info is available, use it for context
    if schema_info is None:
        # Create a simple dictionary from the dataframe
        anomaly_data = []
        for idx, row in anomalies_df.iterrows():
            item = {
                'invoice_id': str(row.get('Invoice_No_', str(idx))),
                'timestamp': str(row.get('Timestamp', '')),
                'anomaly_type': 'Unknown',
                'severity': str(row.get('severity', 'Medium')),
                'fields': {}
            }
            
            # Determine anomaly type
            if 'tax_calculation_error' in anomalies_df.columns and row.get('tax_calculation_error'):
                item['anomaly_type'] = 'Tax Calculation Error'
            elif 'price_qty_mismatch' in anomalies_df.columns and row.get('price_qty_mismatch'):
                item['anomaly_type'] = 'Price Quantity Mismatch'
            elif 'significant_price_reduction' in anomalies_df.columns and row.get('significant_price_reduction'):
                item['anomaly_type'] = 'Significant Price Reduction'
            elif 'high_discount' in anomalies_df.columns and row.get('high_discount'):
                item['anomaly_type'] = 'High Discount'
            elif 'cancelled_with_charges' in anomalies_df.columns and row.get('cancelled_with_charges'):
                item['anomaly_type'] = 'Cancelled with Charges'
            elif 'zomato_status_mismatch' in anomalies_df.columns and row.get('zomato_status_mismatch'):
                item['anomaly_type'] = 'Zomato Status Mismatch'
            elif 'swiggy_status_mismatch' in anomalies_df.columns and row.get('swiggy_status_mismatch'):
                item['anomaly_type'] = 'Swiggy Status Mismatch'
            
            # Add relevant fields
            for col in ['Price', 'Qty_', 'Sub_Total', 'Discount', 'Tax', 'Final_Total',
                        'Status', 'Payment_Type', 'Order_Type', 'discount_percentage']:
                if col in row and not pd.isna(row[col]):
                    item['fields'][col] = str(row[col])
            
            # Add modification details if available
            if 'amount_from' in row and 'amount_to' in row and not pd.isna(row['amount_from']) and not pd.isna(row['amount_to']):
                item['fields']['modified_from'] = str(row['amount_from'])
                item['fields']['modified_to'] = str(row['amount_to'])
                item['fields']['modification_amount'] = str(row.get('modification_amount', row['amount_to'] - row['amount_from']))
            
            anomaly_data.append(item)
    else:
        # Use schema info for more detailed context
        # Implementation would depend on schema format
        anomaly_data = []  # Placeholder
    
    print(f"Prepared {len(anomaly_data)} anomaly records for API")
    return anomaly_data


# Main execution function
def build_and_compare_models(processed_data_path='processed_data.csv'):
    """
    Main function to build and compare anomaly detection models
    """
    # Load preprocessed data
    df, anomaly_flags = load_preprocessed_data(processed_data_path)
    
    # Prepare features
    features_df, numeric_cols, categorical_cols, y = prepare_features(df)
    
    # Create preprocessor
    preprocessor = create_preprocessor(numeric_cols, categorical_cols)
    
    # Transform data
    print("\nTransforming features...")
    X_transformed = preprocessor.fit_transform(features_df)
    print(f"Transformed data shape: {X_transformed.shape}")
    
    # Apply dimensionality reduction for visualization
    print("\nApplying PCA for visualization...")
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_transformed)
    
    # Plot PCA
    plt.figure(figsize=(10, 8))
    if y is not None:
        plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='viridis', alpha=0.5)
        plt.title('PCA Visualization of Data with Anomalies')
    else:
        plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5)
        plt.title('PCA Visualization of Data')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.savefig('pca_visualization.png')
    plt.close()
    
    # Train unsupervised models
    unsupervised_results = train_unsupervised_models(X_transformed, y)
    
    # Train supervised model if target is available
    supervised_result = None
    if y is not None:
        supervised_result = train_supervised_model(X_transformed, y, features_df)
    
    # Compare models
    comparison_results = compare_models(unsupervised_results, supervised_result)
    
    # Determine best model based on F1 score
    best_model_name = comparison_results.sort_values('F1 Score', ascending=False).iloc[0]['Model']
    print(f"\nBest model based on F1 Score: {best_model_name}")
    
    if best_model_name in unsupervised_results:
        best_model_results = unsupervised_results[best_model_name]
    else:
        best_model_results = supervised_result
    
    # Analyze anomalies from the best model
    anomalies_df = analyze_anomalies(df, best_model_results, features_df, numeric_cols, categorical_cols)
    
    # Save models
    models_to_save = {
        'preprocessor': preprocessor,
        'unsupervised_models': {k: v['model'] for k, v in unsupervised_results.items()},
        'supervised_model': supervised_result['model'] if supervised_result else None,
        'numeric_cols': numeric_cols,
        'categorical_cols': categorical_cols
    }
    save_models(models_to_save)
    
    # Create API-ready data
    anomaly_data = create_anomaly_data_for_api(anomalies_df)
    
    # Save anomaly data for API
    with open('anomaly_data_for_api.json', 'w') as f:
        import json
        json.dump(anomaly_data, f, default=str)
    print("Saved anomaly data for API to 'anomaly_data_for_api.json'")
    
    return {
        'models': models_to_save,
        'comparison': comparison_results,
        'best_model': best_model_name,
        'anomalies': anomalies_df,
        'api_data': anomaly_data
    }


# Run the model building process if executed directly
if __name__ == "__main__":
    # Replace with your processed data path if different
    results = build_and_compare_models('/kaggle/working/processed_data.csv')
    
    print("\n--- Model Building Summary ---")
    print(f"Best model: {results['best_model']}")
    print(f"Total anomalies detected: {len(results['anomalies'])}")
    print(f"API data prepared: {len(results['api_data'])} records")
    print("All models saved and ready for API deployment")

Loading preprocessed data...
Loaded 205888 records with 192 features
Found 11 anomaly flag columns: ['high_discount', 'status_z_mismatch', 'status_s_mismatch', 'cancelled_with_charges', 'price_qty_mismatch', 'significant_price_reduction', 'zomato_delivery_minutes_anomaly', 'swiggy_delivery_minutes_anomaly', 'zomato_status_mismatch', 'swiggy_status_mismatch', 'any_basic_anomaly']
Records with at least one anomaly: 18450 (8.96%)

Preparing features for modeling...
Selected Price (missing: 0.00%)
Selected Qty_ (missing: 0.00%)
Selected Sub_Total (missing: 0.00%)
Selected Discount (missing: 0.00%)
Selected Tax (missing: 0.00%)
Selected Final_Total (missing: 0.00%)
Selected discount_percentage (missing: 0.00%)
Dropped modification_amount due to high missingness: 96.23%
Dropped zomato_delivery_minutes due to high missingness: 96.24%
Dropped swiggy_delivery_minutes due to high missingness: 98.69%
Selected CGST_Amount (missing: 0.00%)
Selected SGST_Amount (missing: 0.00%)
Selected VAT_Amount (